In [1]:
# Import packages and set configuration
import pandas as pd
from IPython.display import display

%load_ext autoreload
%autoreload 2

### Asset list
Market Cap Ranking Comparison Based on Historical CoinMarketCap Snapshots.

In [2]:
# Load tickers
tickers = ["BTCUSDT", "ETHUSDT", "BNBUSDT", "SOLUSDT", "XRPUSDT", "ADAUSDT", "AVAXUSDT", "DOGEUSDT", "TRXUSDT",
           "DOTUSDT", "LINKUSDT", "SHIBUSDT", "LTCUSDT", "BCHUSDT", "UNIUSDT"]
pd.read_excel('../data/asset_list.xlsm', dtype={'07.01.2024': 'Int64', '05.10.2025': 'Int64'}).iloc[:15, :2]

,Name,Symbol
0,Bitcoin,BTC
1,Ethereum,ETH
2,BNB,BNB
3,Solana,SOL
4,XRP,XRP
5,Cardano,ADA
6,Avalanche,AVAX
7,Dogecoin,DOGE
8,TRON,TRX
9,Polkadot,DOT


### Strategy 1: Distance Approach (Gatev et al. 2006)
#### Pair selection
-  ##### 2024-01-01 - 2024-07-01 (6 months)
-  ##### Min. SSD of cumulative returns
#### Trading
-  ##### 2024-07-01 - 2024-08-01 (1 month)
-  ##### Spread = normPriceX - normPriceY
-  ##### Std From 6-Month Training (Const in Test Period)
-  ##### Entry Threshold: 2 * Std
-  ##### Exit Threshold: 0 (mean)

In [33]:
training_start = "2024-01-01"
training_end = "2024-07-01"
trading_end = "2024-08-1"
interval = "1h"

In [34]:
# Load data
from modules.data_services.data_pipeline import load_data

training_df = load_data(
    tickers=tickers,
    start=training_start,
    end=training_end,
    interval=interval
)

In [35]:
# Pair Selection
from modules.pair_selection.statistical_tests import sum_of_standard_deviation

ssd_df = sum_of_standard_deviation(training_df)
pairs_1 = ssd_df.iloc[0:1, 0].tolist()
display(ssd_df.head(5))

,pair,ssd
0,ADAUSDT-DOTUSDT,18.816050
1,XRPUSDT-DOTUSDT,31.270303
2,XRPUSDT-ADAUSDT,38.727609
3,BTCUSDT-ETHUSDT,38.812768
4,AVAXUSDT-DOTUSDT,60.430387


,pair,ssd
0,ADAUSDT-DOTUSDT,18.816050
1,XRPUSDT-DOTUSDT,31.270303
2,XRPUSDT-ADAUSDT,38.727609
3,BTCUSDT-ETHUSDT,38.812768
4,AVAXUSDT-DOTUSDT,60.430387


In [36]:
# Load pair
from modules.data_services.data_pipeline import load_pair

data = load_pair(x=pairs_1[0].split('-')[0], y=pairs_1[0].split('-')[1], start=training_start, end=trading_end, interval=interval)

In [37]:
fee_rate = 0.0005  # 0.05%
initial_cash = 100000
position_size = 1  # always 100% of portfolio

entry_threshold = 2 # 2 std from spread
exit_threshold = 0 # mean
stop_loss = 1.5 #1.5 of entry spread; 150%

In [38]:
# Run strategy
from modules.performance.strategy_1 import run_strategy

data.fee_rate = fee_rate
data.initial_cash = initial_cash

run_strategy(data, entry_threshold, exit_threshold, stop_loss, position_size)
data.data[4368:]

KeyboardInterrupt: 

KeyboardInterrupt: 

In [32]:
# Calculate statistics
from modules.performance.strategy_1 import calculate_stats

data.stats = calculate_stats(data)
data.stats

,0% fee,0.05% fee
metric,,
total_return,0.095066,0.094018
cagr,0.000000,0.000000
volatility,0.005928,0.005932
volatility_annual,0.554854,0.555179
max_drawdown,-0.141866,-0.141925
win_count,1.000000,1.000000
lose_count,0.000000,0.000000
win_rate,1.000000,1.000000
max_win,0.095066,0.094518


,0% fee,0.05% fee
metric,,
total_return,0.095066,0.094018
cagr,0.000000,0.000000
volatility,0.005928,0.005932
volatility_annual,0.554854,0.555179
max_drawdown,-0.141866,-0.141925
win_count,1.000000,1.000000
lose_count,0.000000,0.000000
win_rate,1.000000,1.000000
max_win,0.095066,0.094518
